# Counting things in DPLA with SparkSQL

Inspired by Corey Harper's [DPLA Analytics](https://github.com/chrpr/dpla-analytics) work, I wanted to get my feet wet working with basic [DPLA metadata structures](http://dp.la/info/developers/map/) using [SparkSQL](http://spark.apache.org/sql/).  This brings together a few things I haven't done before:  working with DPLA metadata from their [bulk downloads](http://dp.la/info/developers/download/), using SparkSQL, and putting the two together.

In Corey's [Statistical DPLA: Metadata Counting and Word Analysis](https://open.library.ubc.ca/cIRcle/collections/55474/items/1.0220818) from the fall 2015 [DLF Forum](https://www.diglib.org/forums/2015forum/livestream-schedule/) he discusses the results of his work to characterize DPLA records at the scale of "all of it", and to begin to tie that into patterns of use.  In an offline discussion he indicated that one barrier to being able to move faster with this work is the processing time it takes to prep the data to make it amenable to the kind of aggregate summarizations he is performing.  Working at a scale of millions of records, some data prep operations can take hours to complete on a single laptop.

These days, I'll readily admit, I've been walking around with Jupyter and Spark hammers, and this problem sounds very much like a Spark-friendly nail.  There are many other approaches to addressing this particular performance problem, but I have grown to enjoy Spark's terrific performance, friendly APIs, and rapid improvements and am always looking to find more ways to work with it.

With that in mind, in this notebook I'll demonstrate the basics of acquiring a DPLA JSON dump, prepping it to load into Spark, configuring Jupyter/IPython to connect to Spark, loading the JSON into a schema within Spark, and issuing basic queries.  For the sake of expediency, I'll do this using a small subset of DPLA data; in a later notebook I'll pull up a big server and work with the whole dataset to go a little deeper with the data.

## Acquiring DPLA JSON data

Looking into the most recent [DPLA bulk downloads](http://dp.la/info/developers/download/?prefix=2016/02/), we find a 46.8MB (compressed) record set from the Getty called [getty.json.gz](https://dpla-provider-export.s3.amazonaws.com/2016/02/getty.json.gz).  Download that to your local disk:

In [3]:
!wget https://dpla-provider-export.s3.amazonaws.com/2016/02/getty.json.gz

--2016-02-29 13:33:48--  https://dpla-provider-export.s3.amazonaws.com/2016/02/getty.json.gz
Resolving dpla-provider-export.s3.amazonaws.com... 54.231.81.80
Connecting to dpla-provider-export.s3.amazonaws.com|54.231.81.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49112071 (47M) [application/gzip]
Saving to: 'getty.json.gz'

getty.json.gz       100%[===================>]  46.84M  1.20MB/s    in 40s     

2016-02-29 13:34:29 (1.16 MB/s) - 'getty.json.gz' saved [49112071/49112071]



Let's see just how many records that contains (note: ```gzcat``` because I'm on OSX; you might want ```zcat``` instead):

In [4]:
!gzcat getty.json.gz | wc -l

   95910


Looks like over 95,000 records.  Great.  Onto the next step.

## Prepping DPLA JSON data to load into Spark

Spark prefers to load line-oriented JSON.  Line-oriented JSON looks like this:

```
{'id': 1, 'data': ...}
{'id': 2, 'data': ...}
{'id': 3, 'data': ...}
... more ...
```

...which is to say "one record per line".  This makes it amenable to piping and streaming operations.  So far so good.

Note, though, that the [DPLA data doesn't come like that](https://digitalpubliclibraryofamerica.atlassian.net/wiki/display/TECH/Database+export+files) -- rather, bulk download files like the one we just grabbed look more like this:

```
[
    {
        ...
        "_source": { ... record ... }
        ...
    },
    ... more ...
]
```

And that's with a little whitespace added for clarity.  Removing that extra whitespace, they actually look like this:

```
[
{...,"_source": { ... record ... },...}
,{...,"_source": { ... record ... },...}
,{...,"_source": { ... record ... },...}
... more ...
]
```

Note that this is a JSON serialization of a set of records, not line-oriented JSON where each line contains one record.  To work with this data, a JSON deserializer will have to read the entire set into memory before proceeding.  This isn't really a problem, it's just a slightly different approach, and not the one Spark prefers.  Fortunately, we can easily change the recordset-oriented data into line-oriented records.  There are two issues we have to deal with, first being the opening and closing square brackets, and the second being the leading commas.  

Unix to the rescue here.  We can use ```head```, ```tail```, and ```sed``` to handle it.  We just need to know how many lines there are total so we know how many to include.  And we already know that from the above:  95,910.  This implies (skipping the first and last lines) that there are actually 95,908 records.

In [5]:
!gzcat getty.json.gz | head -95909 | tail -95908 > getty-nobrackets.json

And to get rid of the leading commas:

In [6]:
!sed -e 's/^,//' getty-nobrackets.json > getty-lines.json

In [7]:
!wc -l getty-lines.json

   95908 getty-lines.json


Looks right.  On to the next step.

## Configuring Jupyter/IPython to connect to Spark

### Get Spark installed

This part can be tricky, but I think I've got it boiled down to just a few steps.  First, I'm assuming you have Spark installed.  If you don't, it's not hard.  On OSX with [homebrew](http://brew.sh):

```
% brew install apache-spark
```

On other platforms, you might need to [download Spark](http://spark.apache.org/downloads.html) and install it.  The bundles pre-built for Hadoop make it very easy.  Just unwrap the bundle and follow the instructions - you'll be surprised how easy it is to get started.  We're going to use ```pyspark```, which is found in the ```bin``` directory of the Spark distribution you've grabbed.

At this point, I wave my hands, and assume that you are able to type...:

```
% pyspark
```

...and see a few dozen INFO lines scroll by, ending with this:

```
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.11 (default, Feb 15 2016 13:40:02)
SparkContext available as sc, HiveContext available as sqlContext.
>>>
```

And at this point, you can type ```sqlContext```, and should get the following output:

```
>>> sqlContext
<pyspark.sql.context.HiveContext object at 0x10c1d1d10>
```

If that's what you get, you're all set!

(Note that bit about ```sc``` and ```sqlContext``` - we'll use the latter for SparkSQL in a moment, and the former gives you access to anything else non-SQL in Spark.

### Get Jupyter installed

The easiest way to get the Jupyter notebook installed is through Continuum's [Anaconda Python package](https://www.continuum.io/downloads).  Follow their instructions and you'll get a fully working scientific Python environment in a few minutes.

### Connect Jupyter to Spark

I found [these instructions](http://stackoverflow.com/questions/33064031/link-spark-with-ipython-notebook/33065359#33065359) to be the simplest way to connect Jupyter to Spark.  Rather than make you parse them line-by-line, here are the key bits:

```
% export PATH=/WHERE/YOU/PUT/SPARK/bin:$PATH
% export PYSPARK_DRIVER_PYTHON=jupyter
% export PYSPARK_DRIVER_PYTHON_OPTS='notebook' pyspark
```

A few notes on this:  first, substitute where you actually unwrapped Spark for ```/WHERE/YOU/PUT/SPARK```; second, the notebook is now called "Jupyter", not "ipython", so setting ```PYSPARK_DRIVER_PYTHON``` to ```jupyter``` instead of ```ipython``` will save a few warnings; finally, the last bit assures you that whenever you start ```pyspark``` (like you did above) from now on, it will start and open a new Jupyter notebook for you, rather than the command line shell you saw before.

If you like all that, you should add those three lines to your ```.profile``` so your shell will be configured to do it by default, as that Stack Overflow answer suggests.  Doing it the way I listed it here means it only works during your current shell session.  It's up to you.

Test it all out.  Set those environment variables and then type ```pyspark```:

```
% pyspark
```

Your web browser should pop open and a new pyspark notebook should be available.  In the first cell, enter ```sqlContext```, execute it, and you should see the same kind of output you saw above when you did the same thing on the command line.  It should look like this:

In [9]:
sqlContext

Got it?  Good.

## Loading DPLA JSON data into Spark

Getting the line-oriented data into Spark is easier that you think.  It might take a minute or two, though, so don't panic.

In [8]:
getty = sqlContext.read.json("getty-lines.json")

That's it.  The data is now ready for Spark to process.

SparkSQL first will infer a schema from the JSON as it appeared in the record set.  We can examine the schema:

In [10]:
getty.printSchema()

root
 |-- _id: string (nullable = true)
 |-- _index: string (nullable = true)
 |-- _score: long (nullable = true)
 |-- _source: struct (nullable = true)
 |    |-- @context: string (nullable = true)
 |    |-- @id: string (nullable = true)
 |    |-- @type: string (nullable = true)
 |    |-- _id: string (nullable = true)
 |    |-- _rev: string (nullable = true)
 |    |-- admin: struct (nullable = true)
 |    |    |-- sourceResource: struct (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |    |    |-- valid_after_enrich: boolean (nullable = true)
 |    |    |-- validation_message: string (nullable = true)
 |    |-- aggregatedCHO: string (nullable = true)
 |    |-- dataProvider: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- ingestDate: string (nullable = true)
 |    |-- ingestType: string (nullable = true)
 |    |-- ingestionSequence: long (nullable = true)
 |    |-- isShownAt: string (nullable = true)
 |    |-- object: string (nullable = t

Next, to tell SparkSQL that we're going to be using these records in SQL queries for the rest of our session:

In [11]:
getty.registerTempTable("getty")

And now we're ready to issue some SQL queries.

## Issuing basic queries using SparkSQL

From here on out, it's pretty easy - if you know SQL, you're pretty much ready to go.  The key thing to remember, though, is that SparkSQL results come out in a [DataFrame](http://spark.apache.org/docs/latest/sql-programming-guide.html#dataframes) and you'll have to use DataFrame conventions to view them.  Also, remember that every Spark processing job has a little overhead.  You pay the price of a slow setup to gain the benefit of huge speedups Spark provides by running your job in memory and with multiple CPUs.  We'll see this payoff more when we're working with much more on a beefy machine, but in the meantime, just sit tight and wait for the job to run.

In [13]:
count = sqlContext.sql("SELECT COUNT(*) AS the_count FROM getty")
count.show()

+---------+
|the_count|
+---------+
|    95908|
+---------+



Looks right!

For something a little more interesting, let's group and count by object type.  Here we run into another issue we inherit from DPLA's metadata structure:  field names starting with ```@``` and ```_``` should be escaped using backticks.

In [18]:
types = sqlContext.sql("""
SELECT `_source`.sourceResource.type AS type, COUNT(*) AS the_count 
FROM getty 
GROUP BY `_source`.sourceResource.type
ORDER BY the_count DESC
""")
types.show()

+----------------+---------+
|            type|the_count|
+----------------+---------+
|           image|    92190|
|["image","text"]|     2616|
|            text|     1094|
|            null|        3|
|    moving image|        3|
| physical object|        1|
|           sound|        1|
+----------------+---------+



Not too surprisingly for materials from the Getty, most of the records are some form of image.

Ah, here's an issue - some records have multi-valued results.  We'd need to flatten that to count more carefully, let's revisit that later.

For now, thought, let's try another, looking more closely at dates.

In [23]:
dates = sqlContext.sql("""
SELECT `_source`.sourceResource.date.displayDate, COUNT(*) AS count
FROM getty
GROUP BY `_source`.sourceResource.date.displayDate
ORDER BY count DESC
LIMIT 20
""")
dates.show()

+------------+-----+
| displayDate|count|
+------------+-----+
|   1960-1990|73646|
|      [187-]|  718|
|      [188-]|  431|
|        1951|  306|
|      [186-]|  291|
|        1949|  277|
|        1953|  274|
|c. 1675-1725|  269|
|c. 1500-1525|  260|
|        1954|  250|
|        1950|  246|
|        1952|  235|
|        1948|  209|
|        1956|  204|
|        1961|  199|
|        null|  199|
|        1963|  192|
|        1955|  190|
|        1964|  185|
|c. 1700-1725|  184|
+------------+-----+



Ah, bibliographic metadata with its uncertainties and syntax.  Ain't it a joy?

### Processing notes

On my machine these queries took a number of seconds to execute.  With Spark, it's often that last bit -- here, the ```.show()``` call -- that forces Spark to execute the commands you've prepped it to perform.  In this case, the SQL command isn't executed until the ```show()``` command is given.  This allows you to build pipelined operations on DataFrames, which we can get into more some other time.

One of the fun things with Spark is to see what's going on under the hood.  It has its own web interface.  If you're running Spark on your local machine like me, you can visit [localhost:4040](http://localhost:4040) and see the interface for yourself.  Click on the most recently-completed job, view the event timeline, open the DAG visualization, and click further into each stage to see how Spark handled it all.  What you'll see there is why Spark is one of my favorite hammers these days - it'll handle all that complex job tasking for you, and all you have to do is issue a few declarative (in the case of SQL, or functional in a lot of other cases) commands.  Spark will spread those jobs over all the hardware it has available, and pull all the results back together for you too.

Looking at that last job, on my very old laptop (a late 2011 MacBook Air) it took about 20 seconds to process that last query.  If you compare it to what you can get from a good database like MySQL, PostgreSQL, Oracle, or SQL Server, that's terrible.  Really, it's awful.  But think about what you didn't have to do:  define a schema, convert the JSON data, write a custom importer, define and build indexes.  For some quick processing of 90,000 records, the time savings in avoiding those steps can make it worthwhile to go the Spark route.  And if you follow a similar process but on a better machine with more RAM, more CPUs, and faster RAM and CPU than what I have, you get a lot more speed on larger jobs just by having better hardware.  Of course you can also take advantage of similar benefits from good RDBMS systems, but the balance of your time investment will vary.

In any case, it's good to have multiple options.